# FluidityNonlocal Flow Curve: Shear Banding Detection

## Learning Objectives

1. Understand spatial diffusion via cooperativity length ξ in thixotropic fluids
2. Detect shear banding from fluidity profiles (CV > 0.3, f_max/f_min > 10)
3. Compare local vs non-local model predictions on emulsion flow curves
4. Fit 10-parameter nonlocal model using NLSQ → NUTS workflow
5. Quantify when non-local effects dominate vs local approximation suffices

## Model Overview

**FluidityNonlocal** extends the local fluidity model with spatial diffusion:

$$
\frac{\partial f}{\partial t} = \frac{f_{\text{loc}}(\sigma) - f}{\theta} + \xi^2 \frac{\partial^2 f}{\partial y^2}
$$

where:
- $f_{\text{loc}}(\sigma) = f_0 + (f_1 - f_0) \tanh\left(\frac{\sigma - \sigma_c}{\Delta\sigma}\right)$ (local steady-state solution)
- $\theta$ = structural relaxation time
- $\xi$ = cooperativity length (spatial diffusion scale)
- $y$ = gap coordinate (0 to $h$)

**Shear Banding Metrics:**
- Coefficient of variation: $\text{CV} = \sigma(f) / \mu(f) > 0.3$
- Fluidity contrast: $f_{\max} / f_{\min} > 10$

**Parameters (10):**
1. $f_0$ = low-stress fluidity (s⁻¹)
2. $f_1$ = high-stress fluidity (s⁻¹)
3. $\sigma_c$ = critical stress (Pa)
4. $\Delta\sigma$ = transition width (Pa)
5. $\theta$ = relaxation time (s)
6. $n$ = flow index
7. $\alpha$ = stress exponent
8. $K$ = consistency (Pa·sⁿ)
9. $\tau_y$ = yield stress (Pa)
10. $\xi$ = cooperativity length (m)

In [1]:
# Colab setup
try:
    import google.colab
    IN_COLAB = True
    !pip install -q rheojax nlsq numpyro arviz
except ImportError:
    IN_COLAB = False

# Standard imports
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# RheoJAX imports
from rheojax.core.jax_config import safe_import_jax
from rheojax.core.data import RheoData
from rheojax.models.fluidity import FluidityNonlocal, FluidityLocal
from rheojax.logging import configure_logging, get_logger
from rheojax.utils.metrics import compute_fit_quality

# JAX setup (NLSQ auto-configures float64)
jax, jnp = safe_import_jax()

# Logging
configure_logging(level="INFO")
logger = get_logger(__name__)

# Plotting defaults
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'font.size': 11,
    'axes.labelsize': 12,
    'axes.titlesize': 13,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
})

logger.info("Setup complete", jax_version=jax.__version__)

In [2]:
def compute_fit_quality(y_true, y_pred):
    """Compute R² and RMSE."""
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    residuals = y_true - y_pred
    if y_true.ndim > 1:
        residuals = residuals.ravel()
        y_true = y_true.ravel()
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    r2 = 1.0 - ss_res / ss_tot if ss_tot > 0 else 0.0
    rmse = np.sqrt(np.mean(residuals**2))
    return {'R2': r2, 'RMSE': rmse}

## Theory: Non-Local Effects and Shear Banding

### Spatial Diffusion PDE

The fluidity field $f(y, t)$ evolves according to:

$$
\frac{\partial f}{\partial t} = \underbrace{\frac{f_{\text{loc}}(\sigma) - f}{\theta}}_{\text{Local relaxation}} + \underbrace{\xi^2 \frac{\partial^2 f}{\partial y^2}}_{\text{Spatial diffusion}}
$$

**Cooperativity Length $\xi$:**
- Physical interpretation: length scale over which stress perturbations propagate
- Typical values: 1-100 μm for soft materials
- $\xi \ll h$: local approximation valid (FluidityLocal sufficient)
- $\xi \sim h$: non-local effects important (shear banding possible)

### Shear Banding Criterion

Steady-state fluidity profile $f(y)$ indicates banding when:

1. **High spatial variation:**
   $$\text{CV} = \frac{\sigma_f}{\mu_f} > 0.3$$
   where $\sigma_f = \sqrt{\langle (f - \langle f \rangle)^2 \rangle}$

2. **Strong contrast:**
   $$\frac{f_{\max}}{f_{\min}} > 10$$

3. **Bimodal distribution:**
   - Low-fluidity band: $f \approx f_0$ (nearly solid)
   - High-fluidity band: $f \approx f_1$ (flowing)

### Constitutive Relation

Stress-fluidity coupling:

$$
\sigma = \tau_y + K \dot{\gamma}^n, \quad \dot{\gamma} = f \cdot \sigma^\alpha
$$

**Key Difference from Local Model:**
- Local: $f$ uniform across gap → single shear rate
- Nonlocal: $f(y)$ profile → shear rate banding $\dot{\gamma}(y)$

## Data: Emulsion Flow Curve

Synthetic emulsion data showing characteristic stress plateau (yield region).

In [3]:
# Generate synthetic emulsion flow curve data
np.random.seed(42)

# Shear rate range spanning yield transition
gamma_dot = np.logspace(-3, 2, 50)  # 0.001 to 100 s^-1

# Ground truth parameters (moderate shear banding)
f0_true = 0.01      # Low fluidity (s^-1)
f1_true = 1.0       # High fluidity (s^-1)
sigma_c_true = 50.0 # Critical stress (Pa)
delta_sigma_true = 10.0  # Transition width (Pa)
theta_true = 5.0    # Relaxation time (s)
n_true = 0.5        # Shear-thinning
alpha_true = 1.0    # Linear stress dependence
K_true = 20.0       # Consistency (Pa·s^n)
tau_y_true = 30.0   # Yield stress (Pa)
xi_true = 5e-5      # 50 μm cooperativity length

# Simple Herschel-Bulkley flow curve for synthetic data
# sigma = tau_y + K * gamma_dot^n
sigma_true = tau_y_true + K_true * gamma_dot**n_true

# Add measurement noise (5% relative + 2 Pa absolute)
noise = 0.05 * sigma_true + 2.0 * np.random.randn(len(sigma_true))
sigma = sigma_true + noise

# Ensure positive stresses
sigma = np.maximum(sigma, 1.0)

# Visualization
fig, ax = plt.subplots()
ax.loglog(gamma_dot, sigma, 'o', label='Measured', alpha=0.7)
ax.loglog(gamma_dot, sigma_true, '-', label='True (no noise)', linewidth=2)
ax.axhline(sigma_c_true, color='gray', linestyle='--', label=f'σ_c = {sigma_c_true} Pa')
ax.set_xlabel('Shear Rate $\dot{\gamma}$ (s$^{-1}$)')
ax.set_ylabel('Stress $\sigma$ (Pa)')
ax.set_title('Emulsion Flow Curve (Synthetic Data)')
ax.legend()
ax.grid(True, which='both', alpha=0.3)
plt.tight_layout()
plt.show()
plt.close('all')

logger.info("Data generated", n_points=len(gamma_dot), sigma_range=(sigma.min(), sigma.max()))

<>:35: SyntaxWarning: invalid escape sequence '\d'
<>:36: SyntaxWarning: invalid escape sequence '\s'
<>:35: SyntaxWarning: invalid escape sequence '\d'
<>:36: SyntaxWarning: invalid escape sequence '\s'
/var/folders/rq/lgpr8fvj349cr_wvkzv99md80000gp/T/ipykernel_66433/1375425042.py:35: SyntaxWarning: invalid escape sequence '\d'
  ax.set_xlabel('Shear Rate $\dot{\gamma}$ (s$^{-1}$)')
/var/folders/rq/lgpr8fvj349cr_wvkzv99md80000gp/T/ipykernel_66433/1375425042.py:36: SyntaxWarning: invalid escape sequence '\s'
  ax.set_ylabel('Stress $\sigma$ (Pa)')
/var/folders/rq/lgpr8fvj349cr_wvkzv99md80000gp/T/ipykernel_66433/1375425042.py:41: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Model Initialization

Configure **FluidityNonlocal** with spatial discretization:
- `N_y = 64`: number of grid points across gap
- `gap_width = 1e-3`: 1 mm gap (typical rheometer geometry)

In [4]:
# Create RheoData object
rheo_data = RheoData(
    x=gamma_dot,
    y=sigma,
    initial_test_mode='flow_curve',
    metadata={'material': 'emulsion', 'temperature': 25.0}
)

# Initialize nonlocal model
model_nonlocal = FluidityNonlocal(
    N_y=64,           # Spatial resolution
    gap_width=1e-3    # 1 mm gap
)

logger.info(
    "Model initialized",
    n_params=len(model_nonlocal.parameters),
    N_y=64,
    gap_width=1e-3
)

# Display parameter bounds
print("\nParameter Bounds:")
for name, param in model_nonlocal.parameters.items():
    print(f"  {name:12s}: [{param.bounds[0]:8.2e}, {param.bounds[1]:8.2e}]")


Parameter Bounds:
  G           : [1.00e+03, 1.00e+09]
  tau_y       : [1.00e+01, 1.00e+06]
  K           : [1.00e+00, 1.00e+06]
  n_flow      : [1.00e-01, 2.00e+00]
  f_eq        : [1.00e-12, 1.00e-03]
  f_inf       : [1.00e-06, 1.00e+00]
  theta       : [1.00e-01, 1.00e+04]
  a           : [0.00e+00, 1.00e+02]
  n_rejuv     : [0.00e+00, 2.00e+00]
  xi          : [1.00e-09, 1.00e-03]


## NLSQ Fitting

Non-linear least squares optimization using NLSQ 0.6.6+ workflow system.

In [5]:
# NLSQ fit (warm-start for Bayesian inference)
model_nonlocal.fit(gamma_dot, sigma, test_mode='flow_curve', method='scipy')

# Extract fitted parameters
params_nlsq = {
    name: model_nonlocal.parameters[name].value
    for name in model_nonlocal.parameters.keys()
}

print("\nNLSQ Fitted Parameters:")
for name, value in params_nlsq.items():
    print(f"  {name:12s}: {value:10.4e}")

# Predictions
sigma_pred_nlsq = model_nonlocal.predict(gamma_dot, test_mode='flow_curve')

# Compute fit quality
metrics = compute_fit_quality(sigma, np.array(sigma_pred_nlsq).flatten())
r_squared = metrics['R2']
rmse = metrics['RMSE']

print(f"\nR² = {r_squared:.6f}")
print(f"RMSE = {rmse:.4f} Pa")

# Plot fit quality
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Flow curve
ax1.loglog(gamma_dot, sigma, 'o', label='Data', alpha=0.7)
ax1.loglog(gamma_dot, sigma_pred_nlsq, '-', label='NLSQ Fit', linewidth=2)
ax1.set_xlabel(r'Shear Rate $\dot{\gamma}$ (s$^{-1}$)')
ax1.set_ylabel(r'Stress $\sigma$ (Pa)')
ax1.set_title(f'NLSQ Fit (R² = {r_squared:.4f})')
ax1.legend()
ax1.grid(True, which='both', alpha=0.3)

# Residuals
residuals = sigma - np.array(sigma_pred_nlsq).flatten()
ax2.semilogx(gamma_dot, residuals, 'o', alpha=0.7)
ax2.axhline(0, color='k', linestyle='--', linewidth=1)
ax2.set_xlabel(r'Shear Rate $\dot{\gamma}$ (s$^{-1}$)')
ax2.set_ylabel(r'Residual $\sigma - \sigma_{\mathrm{pred}}$ (Pa)')
ax2.set_title('Residual Analysis')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
plt.close('all')

logger.info("NLSQ fit complete", R2=r_squared, RMSE=rmse)

05:27:01 | INFO     | rheojax.models.fluidity.nonlocal_model | model_fit started | operation=model_fit | phase=start | model=FluidityNonlocal | test_mode=unknown | data_shape=(50,)


05:27:01 | INFO     | rheojax.utils.optimization | Using SciPy least_squares directly (method='scipy') | n_params=10


05:27:01 | INFO     | rheojax.models.fluidity.nonlocal_model | model_fit completed | operation=model_fit | phase=end | elapsed_seconds=0.1834 | status=success | model=FluidityNonlocal | test_mode=flow_curve | data_shape=(50,) | N_y=64


05:27:01 | INFO     | rheojax.core.base | Fit completed | model=FluidityNonlocal | fitted=True | R2=-327.2350 | data_shape=(50,)



NLSQ Fitted Parameters:
  G           : 1.0000e+06
  tau_y       : 1.4950e+01
  K           : 4.4540e+02
  n_flow      : 4.1947e-01
  f_eq        : 1.0000e-06
  f_inf       : 1.0000e-03
  theta       : 1.0000e+01
  a           : 1.0000e+00
  n_rejuv     : 1.0000e+00
  xi          : 1.0000e-05

R² = -327.235013
RMSE = 953.1024 Pa


/var/folders/rq/lgpr8fvj349cr_wvkzv99md80000gp/T/ipykernel_66433/2137332370.py:47: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Bayesian Inference with NUTS

Use NLSQ solution as warm-start for Hamiltonian Monte Carlo sampling.

In [6]:
# Bayesian inference (4 chains for production-ready diagnostics)
result_bayes = model_nonlocal.fit_bayesian(
    gamma_dot, sigma,
    test_mode='flow_curve',
    num_warmup=1000,
    num_samples=2000,
    num_chains=4,
    seed=42
)

# Extract posterior samples
posterior = result_bayes.posterior_samples

# Compute credible intervals (95%)
intervals = model_nonlocal.get_credible_intervals(posterior, credibility=0.95)

print("\nBayesian Parameter Estimates (95% HDI):")
print(f"{'Parameter':<12s} {'NLSQ':>12s} {'Median':>12s} {'Lower':>12s} {'Upper':>12s}")
print("-" * 60)
for name in model_nonlocal.parameters.keys():
    nlsq_val = params_nlsq[name]
    median = float(jnp.median(posterior[name]))
    lower, upper = intervals[name]
    print(f"{name:<12s} {nlsq_val:12.4e} {median:12.4e} {lower:12.4e} {upper:12.4e}")

logger.info("Bayesian inference complete", num_samples=2000, num_chains=4)

05:27:01 | INFO     | rheojax.core.bayesian | bayesian_inference started | operation=bayesian_inference | phase=start | model=FluidityNonlocal | num_warmup=1000 | num_samples=2000 | num_chains=4


05:27:01 | INFO     | rheojax.core.bayesian | Bayesian inference started | model=FluidityNonlocal | test_mode=flow_curve | num_warmup=1000 | num_samples=2000 | num_chains=4


  0%|          | 0/3000 [00:00<?, ?it/s]

warmup:   0%|          | 1/3000 [00:02<2:01:06,  2.42s/it]

warmup:   0%|          | 2/3000 [00:04<1:59:52,  2.40s/it]

...

sample: 100%|██████████| 3000/3000 [00:18<00:00, 160.04it/s]

05:27:22 | INFO     | rheojax.core.bayesian | Bayesian inference completed | model=FluidityNonlocal | divergences=0 | r_hat_max=1.0000 | ess_min=8000.0000


05:27:22 | INFO     | rheojax.core.bayesian | bayesian_inference completed | operation=bayesian_inference | phase=end | elapsed_seconds=20.6169 | status=success | model=FluidityNonlocal | num_warmup=1000 | num_samples=2000 | num_chains=4 | divergences=0 | r_hat_max=1.0000 | ess_min=8000.0000


05:27:22 | INFO     | rheojax.core.base | Bayesian fit completed | model=FluidityNonlocal | num_warmup=1000 | num_samples=2000 | num_chains=4 | r_hat={'G': 1.0, 'tau_y': 1.0, 'K': 1.0, 'n_flow': 1.0, 'f_eq': 1.0, 'f_inf': 1.0, 'theta': 1.0, 'a': 1.0, 'n_rejuv': 1.0, 'xi': 1.0} | ess={'G': 8000.0, 'tau_y': 8000.0, 'K': 8000.0, 'n_flow': 8000.0, 'f_eq': 8000.0, 'f_inf': 8000.0, 'theta': 8000.0, 'a': 8000.0, 'n_rejuv': 8000.0, 'xi': 8000.0}



Bayesian Parameter Estimates (95% HDI):
Parameter            NLSQ       Median        Lower        Upper
------------------------------------------------------------
G              1.0000e+06   5.0585e+08   3.4881e+07   9.7422e+08
tau_y          1.4950e+01   3.1309e+01   3.0411e+01   3.2255e+01
K              4.4540e+02   2.0723e+01   1.9692e+01   2.1731e+01
n_flow         4.1947e-01   5.0232e-01   4.9057e-01   5.1349e-01
f_eq           1.0000e-06   5.0284e-04   4.9235e-05   9.9465e-04
f_inf          1.0000e-03   4.9589e-01   4.7230e-02   9.9988e-01
theta          1.0000e+01   5.0395e+03   1.4558e+01   9.5505e+03
a              1.0000e+00   5.0502e+01   4.2111e-01   9.4812e+01
n_rejuv        1.0000e+00   1.0007e+00   9.3740e-02   1.9892e+00
xi             1.0000e-05   4.9970e-04   3.7401e-05   9.8837e-04


## ArviZ Diagnostics

Assess MCMC convergence and posterior quality.

In [7]:
import arviz as az

# Convert to InferenceData
idata = az.from_numpyro(result_bayes.mcmc)

# Summary statistics
summary = az.summary(idata, hdi_prob=0.95)
print("\nArviZ Summary:")
print(summary)

# Check convergence
r_hat_max = summary['r_hat'].max()
ess_bulk_min = summary['ess_bulk'].min()
ess_tail_min = summary['ess_tail'].min()

print(f"\nConvergence Diagnostics:")
print(f"  Max R-hat: {r_hat_max:.4f} (target: < 1.01)")
print(f"  Min ESS bulk: {ess_bulk_min:.0f} (target: > 400)")
print(f"  Min ESS tail: {ess_tail_min:.0f} (target: > 400)")

if r_hat_max > 1.01:
    logger.warning("Poor convergence detected", r_hat_max=r_hat_max)
else:
    logger.info("MCMC converged", r_hat_max=r_hat_max, ess_bulk_min=ess_bulk_min)


ArviZ Summary:
                 mean            sd      hdi_2.5%     hdi_97.5%    mcse_mean  \
G        5.018488e+08  2.847479e+08  3.488107e+07  9.742228e+08  2665871.453   
K        2.073100e+01  5.240000e-01  1.969200e+01  2.173100e+01        0.009   
a        5.022900e+01  2.871400e+01  4.210000e-01  9.481200e+01        0.284   
f_eq     1.000000e-03  0.000000e+00  0.000000e+00  1.000000e-03        0.000   
f_inf    4.990000e-01  2.930000e-01  4.700000e-02  1.000000e+00        0.003   
n_flow   5.020000e-01  6.000000e-03  4.910000e-01  5.130000e-01        0.000   
n_rejuv  1.001000e+00  5.750000e-01  9.400000e-02  1.989000e+00        0.006   
sigma    1.947000e+00  2.080000e-01  1.562000e+00  2.351000e+00        0.002   
tau_y    3.130600e+01  4.710000e-01  3.041100e+01  3.225500e+01        0.007   
theta    5.006584e+03  2.948346e+03  1.455800e+01  9.550468e+03       30.822   
xi       1.000000e-03  0.000000e+00  0.000000e+00  1.000000e-03        0.000   

             mcse_sd  e

In [8]:
# Trace plots (check mixing)
az.plot_trace(idata, compact=True, figsize=(12, 10))
plt.tight_layout()
plt.show()

/var/folders/rq/lgpr8fvj349cr_wvkzv99md80000gp/T/ipykernel_66433/4123663942.py:4: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [9]:
# Pair plot (correlations)
# Use actual parameter names from FluidityNonlocal model
param_names = list(model_nonlocal.parameters.keys())[:5]  # First 5 parameters

az.plot_pair(
    idata,
    var_names=param_names,
    kind='hexbin',
    divergences=True,
    figsize=(12, 12)
)
plt.tight_layout()
plt.show()
plt.close('all')

/var/folders/rq/lgpr8fvj349cr_wvkzv99md80000gp/T/ipykernel_66433/1022065861.py:13: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [10]:
# Forest plot (credible intervals)
az.plot_forest(idata, hdi_prob=0.95, figsize=(10, 8))
plt.tight_layout()
plt.show()

/var/folders/rq/lgpr8fvj349cr_wvkzv99md80000gp/T/ipykernel_66433/1292600202.py:4: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Comparison: Nonlocal vs Local Model

Fit **FluidityLocal** to assess whether non-local effects are necessary.

In [11]:
# Initialize local model (no spatial diffusion)
model_local = FluidityLocal()

# NLSQ fit
model_local.fit(gamma_dot, sigma, test_mode='flow_curve', method='scipy')

# Predictions
sigma_pred_local = model_local.predict(gamma_dot, test_mode='flow_curve')

# Compute fit quality
metrics_local = compute_fit_quality(sigma, np.array(sigma_pred_local).flatten())
r_squared_local = metrics_local['R2']

print(f"\nLocal Model R² = {r_squared_local:.6f}")
print(f"Nonlocal Model R² = {r_squared:.6f}")
print(f"Improvement: ΔR² = {r_squared - r_squared_local:.6f}")

# Overlay comparison
fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(gamma_dot, sigma, 'o', label='Data', alpha=0.7, markersize=6)
ax.loglog(gamma_dot, sigma_pred_nlsq, '-', label='Nonlocal (NLSQ)', linewidth=2.5)
ax.loglog(gamma_dot, sigma_pred_local, '--', label='Local (NLSQ)', linewidth=2)
ax.set_xlabel(r'Shear Rate $\dot{\gamma}$ (s$^{-1}$)')
ax.set_ylabel(r'Stress $\sigma$ (Pa)')
ax.set_title('Nonlocal vs Local Model Predictions')
ax.legend()
ax.grid(True, which='both', alpha=0.3)
plt.tight_layout()
plt.show()
plt.close('all')

logger.info(
    "Model comparison complete",
    local_R2=r_squared_local,
    nonlocal_R2=r_squared
)

05:27:24 | INFO     | rheojax.models.fluidity.local | model_fit started | operation=model_fit | phase=start | model=FluidityLocal | test_mode=unknown | data_shape=(50,)


05:27:24 | INFO     | rheojax.utils.optimization | Using SciPy least_squares directly (method='scipy') | n_params=9


05:27:24 | INFO     | rheojax.models.fluidity.local | model_fit completed | operation=model_fit | phase=end | elapsed_seconds=0.0766 | status=success | model=FluidityLocal | test_mode=flow_curve | data_shape=(50,)


05:27:24 | INFO     | rheojax.core.base | Fit completed | model=FluidityLocal | fitted=True | R2=-4.7999e+04 | data_shape=(50,)


/var/folders/rq/lgpr8fvj349cr_wvkzv99md80000gp/T/ipykernel_66433/1737393213.py:29: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()



Local Model R² = -47999.335416
Nonlocal Model R² = -327.235013
Improvement: ΔR² = 47672.100403


## Shear Banding Analysis

Compute fluidity profile statistics to detect shear banding.

In [12]:
# Extract cooperativity length from posterior
xi_samples = posterior['xi']
xi_median = float(jnp.median(xi_samples))
xi_hdi = intervals['xi']

print(f"\nCooperativity Length ξ:")
print(f"  Median: {xi_median*1e6:.2f} μm")
print(f"  95% HDI: [{xi_hdi[0]*1e6:.2f}, {xi_hdi[1]*1e6:.2f}] μm")
print(f"  Gap width: {model_nonlocal.gap_width*1e3:.2f} mm")
print(f"  Ratio ξ/h: {xi_median/model_nonlocal.gap_width:.4f}")

# Shear banding criterion
if xi_median / model_nonlocal.gap_width > 0.01:
    print("\n⚠ Non-local effects significant (ξ/h > 0.01) → potential shear banding")
else:
    print("\n✓ Local approximation valid (ξ/h < 0.01) → minimal shear banding")

# Note: Full fluidity profile f(y) would require spatial simulation
# For flow curve fitting, we use spatially-averaged response
# True banding detection requires startup/creep protocols with spatial resolution


Cooperativity Length ξ:
  Median: 499.70 μm
  95% HDI: [37.40, 988.37] μm
  Gap width: 1.00 mm
  Ratio ξ/h: 0.4997

⚠ Non-local effects significant (ξ/h > 0.01) → potential shear banding


## Save Results

Export NLSQ parameters, Bayesian posteriors, and diagnostics.

In [13]:
# Create output directory
output_dir = Path('../outputs/fluidity/nonlocal/flow_curve')
output_dir.mkdir(parents=True, exist_ok=True)

# Save NLSQ parameters
params_file = output_dir / 'nlsq_parameters.txt'
with open(params_file, 'w') as f:
    f.write("NLSQ Fitted Parameters\n")
    f.write("=" * 40 + "\n\n")
    for name, value in params_nlsq.items():
        f.write(f"{name:12s}: {value:12.6e}\n")
    f.write(f"\nR² = {r_squared:.8f}\n")
    f.write(f"RMSE = {rmse:.6f} Pa\n")

# Save Bayesian summary
summary_file = output_dir / 'bayesian_summary.txt'
with open(summary_file, 'w') as f:
    f.write(summary.to_string())

# Save posterior samples (NetCDF format)
posterior_file = output_dir / 'posterior.nc'
idata.to_netcdf(posterior_file)

# Save comparison plot
fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(gamma_dot, sigma, 'o', label='Data', alpha=0.7, markersize=6)
ax.loglog(gamma_dot, sigma_pred_nlsq, '-', label='Nonlocal', linewidth=2.5)
ax.loglog(gamma_dot, sigma_pred_local, '--', label='Local', linewidth=2)
ax.set_xlabel(r'Shear Rate $\dot{\gamma}$ (s$^{-1}$)')
ax.set_ylabel(r'Stress $\sigma$ (Pa)')
ax.set_title('FluidityNonlocal vs FluidityLocal')
ax.legend()
ax.grid(True, which='both', alpha=0.3)
plt.tight_layout()
plt.savefig(output_dir / 'model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close('all')

print(f"\nResults saved to: {output_dir.absolute()}")
logger.info("Results saved", output_dir=str(output_dir.absolute()))


Results saved to: /Users/b80985/Projects/rheojax/examples/fluidity/../outputs/fluidity/nonlocal/flow_curve


/var/folders/rq/lgpr8fvj349cr_wvkzv99md80000gp/T/ipykernel_66433/1751539841.py:36: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Key Takeaways

### When Non-Local Effects Matter

1. **Cooperativity length ξ:**
   - $\xi/h < 0.01$: Local model sufficient (uniform fluidity)
   - $0.01 < \xi/h < 0.1$: Moderate non-local effects (weak banding)
   - $\xi/h > 0.1$: Strong non-local effects (pronounced shear banding)

2. **Protocol dependence:**
   - **Flow curves**: Steady-state averages $\implies$ local model often adequate
   - **Startup**: Transient banding $\implies$ nonlocal model critical
   - **Creep**: Viscosity bifurcation $\implies$ nonlocal reveals delayed yielding

3. **Material signatures:**
   - **Emulsions**: Moderate ξ (10-50 μm), weak-to-moderate banding
   - **Microgels**: Large ξ (50-200 μm), strong banding
   - **Colloidal glasses**: Small ξ (1-10 μm), nearly local behavior

4. **Computational cost:**
   - Local model: $O(N_{\dot{\gamma}})$ algebraic solve
   - Nonlocal model: $O(N_{\dot{\gamma}} \times N_y \times N_t)$ PDE solve
   - Use local model first; upgrade to nonlocal if $R^2$ improvement $> 0.05$

### Model Selection Workflow

```
1. Fit FluidityLocal → get R²_local
2. Fit FluidityNonlocal → get R²_nonlocal, ξ
3. If ΔR² > 0.05 AND ξ/h > 0.01:
     → Nonlocal effects significant, use FluidityNonlocal
   Else:
     → Local approximation valid, use FluidityLocal
```

### Next Steps

- **Startup simulations**: Reveal stress overshoot and band formation dynamics
- **Creep protocols**: Detect viscosity bifurcation (slow vs fast creep)
- **LAOS analysis**: Nonlinear stress-strain loops with spatial heterogeneity
- **Parameter identifiability**: Joint analysis of multiple protocols to constrain ξ